a notebook to split files out into chips

In [1]:
import cv2
from PIL import Image
import fastai
import torch
import torchvision
import os
import math
import gdal
import numpy as np

In [2]:
#create the data subdirectories and set up data variables (do one class at a time)
data_source_dir = "../summerIce/"
image_list = 251
image_prefix = "SummerIce"
chip_dir = "../summerIce/chips/"

In [3]:
#create chip subdirectory
try:
    # Create target Directory
    os.mkdir(chip_dir)
    print("Directory " , chip_dir ,  " Created ") 
except FileExistsError:
    print("Directory " , chip_dir ,  " already exists")

Directory  ../summerIce/chips/  Created 


In [4]:
os.getcwd()


'/project/ICE/icerepo'

In [5]:
#let me try loading in gdal - https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html

ds = gdal.Open(data_source_dir+image_prefix+"1"+".tif")

In [6]:
#excellent, gdal gets it, now to get into numpy from gdal

myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

In [7]:
myarray.shape

(1630, 1986)

In [12]:
#so modified now that deserves it's own approach. This will go through a sliding window and save each 100x100 as a float64 tensor, save to disk as a tensor file in the chips directory


def slide_window_tensor(image, window_w, window_h, index):
    w, h = image.shape
    w_new, h_new = math.floor(w/window_w), math.floor(h/window_h)
    #print(w,h,w_new,h_new)
    xmin = -30.00
    xmax = -10.00
    for i in range(w_new):
        for j in range(h_new):
            window = image[i*window_w:(i+1)*window_w, j*window_h:(j+1)*window_h]
            #print(window.shape,window)
            #normalize using min=-30 and max=-10 per gee code given to me. xnew = x-xmin/xmax-xmin
            normwindow = (window-xmin)/(xmax-xmin)
            tensor = torch.from_numpy(normwindow)
            #print(tensor)
            torch.save(tensor, chip_dir+image_prefix+str(index)+'.pt')
            index += 1
    return index




In [13]:
#now write the loop that will go through each of the files and output them
index = 0
for i in range(1, image_list):
    ds = gdal.Open(data_source_dir+image_prefix+str(i)+".tif")
    if ds is not None:
        myarray = np.array(ds.GetRasterBand(1).ReadAsArray())
        index = slide_window_tensor(myarray, 100, 100, index)
    print(index)
    

304
646
1007
1279
1602
1810
2228
2298
2298
2304
2310
2312
2312
2312
2313
2314
2314
2314
2314
2314
2314
2318
2319
2319
2319
2320
2321
2325
2333
2334
2335
2337
2337
2338
2339
2343
2347
2348
2348
2349
2361
2363
2369
2370
2371
2372
2384
2386
2390
2390
2391
2397
2409
2424
2433
2439
2442
2450
2465
2477
2502
2544
2568
2592
2628
2658
2693
2713
2719
2728
2764
2799
2820
2828
2840
2860
2880
2904
2928
2944
2953
2962
2974
2990
3002
3022
3026
3030
3034
3038
3039
3045
3049
3057
3057
3063
3067
3073
3082
3091
3103
3107
3123
3143
3149
3158
3162
3170
3182
3188
3192
3198
3207
3208
3209
3225
3237
3243
3249
3305
3317
4100
4408
4872
5192
5936
5976
5980
5981
5990
6120
6132
6168
6204
6216
6231
6232
6234
6234
6238
6244
6286
6288
6300
6301
6302
6303
6303
6304
6313
6325
6327
6331
6332
6332
6333
6337
6337
6340
6342
6348
6357
6382
6386
6395
6413
6425
6434
6440
6442
6454
6466
6470
6476
6482
6491
6507
6542
6554
6574
6586
6586
6588
6588
6592
6601
6621
6621
6623
6687
6705
6720
6784
6809
6851
6876
6912
6952
6956
6968
69

In [ ]:
# don't go here right now, but a more in depth description of unfolding http://jeankossaifi.com/blog/unfolding.html

In [3]:
#we'll iterate on this in a minute, but I want to try on one image first. Load an image
img = Image.open(data_source_dir+image_prefix+"1"+".tif")

UnidentifiedImageError: cannot identify image file '../summerIce/SummerIce1.tif'

In [18]:
#seems like the image is in a weird TIFF format that pillow doesn't like
from PIL import TiffImagePlugin
TiffImagePlugin.DEBUG = True
with open(data_source_dir+image_prefix+"1"+".tif", 'rb') as f:
    TiffImagePlugin.TiffImageFile(f)

*** TiffImageFile._open ***
- __first: 8
- ifh:  b'II*\x00\x08\x00\x00\x00'
Seeking to frame 0, on frame -1, __next 8, location: 8
Loading tags, location: 8
tag: ImageWidth (256) - type: short (3) - value: 1986
tag: ImageLength (257) - type: short (3) - value: 1630
tag: BitsPerSample (258) - type: short (3) - value: (64,)
tag: Compression (259) - type: short (3) - value: 5
tag: PhotometricInterpretation (262) - type: short (3) - value: 1
tag: SamplesPerPixel (277) - type: short (3) - value: 1
tag: PlanarConfiguration (284) - type: short (3) - value: 1
tag: Predictor (317) - type: short (3) - value: 1
tag: TileWidth (322) - type: short (3) - value: 256
tag: TileLength (323) - type: short (3) - value: 256
tag: TileOffsets (324) - type: long (4) Tag Location: 142 - Data Location: 454 - value: <table: 224 bytes>
tag: TileByteCounts (325) - type: long (4) Tag Location: 154 - Data Location: 230 - value: <table: 224 bytes>
tag: SampleFormat (339) - type: short (3) - value: (3,)
tag: ModelPixe

SyntaxError: unknown pixel mode (<string>)

In [7]:
#create a set of sliding windows of size 100x100 from https://stackoverflow.com/questions/6811183/rolling-window-for-1d-arrays-in-numpy/6811241#6811241
def rolling_window(a, window_size):
    shape = (a.shape[0] - window_size + 1, window_size) + a.shape[1:]
    strides = (a.strides[0],) + a.strides
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

newarray = rolling_window(myarray, 100)
newarray.shape

(1531, 100, 1986)

In [13]:
#not sure I understand that. The stride bit is confusing. It's less computationally efficient, but let's try the naive approach here: http://arogozhnikov.github.io/2015/09/30/NumpyTipsAndTricks2.html
import math

def slide_window(image, window_w, window_h):
    w, h = image.shape
    w_new, h_new = math.floor(w/window_w), math.floor(h/window_h)
    print(w,h,w_new,h_new)
    index = 0
    xmin = -30
    xmax = -10
    for i in range(w_new):
        for j in range(h_new):
            window = image[i*window_w:(i+1)*window_w, j*window_h:(j+1)*window_h]
            print(window.shape,window)
            #normalize using min=-30 and max=-10 per gee code given to me. xnew = x-xmin/xmax-xmin
            normwindow = ((window-xmin)/(xmax-xmin))*255
            #this doesn't quite work. Still saving it as a float64 rather than a uint8. Would rather keep the precision, though, so maybe I just save as a tensor?
            uimg = normwindow.astype(np.uint8)
            # this kills the kernel cv2.imshow("window",uimg)
            print(uimg)
            im = Image.fromarray(uimg)
            #im = im.convert("L")
            im.save(chip_dir+image_prefix+str(index)+'.png')
            index += 1
    return    

slide_window(myarray, 100, 100)
#output.shape
#going to create a new variant of this to save as tensors with full precision, won't be easily visualizable but that's probably OK

1630 1986 16 19
(100, 100) [[-15.32095191 -15.32095191 -15.47503733 ... -16.10417079 -16.39201535
  -16.39201535]
 [-15.32095191 -15.32095191 -15.47503733 ... -16.10417079 -16.39201535
  -16.39201535]
 [-15.32095191 -15.32095191 -15.47503733 ... -16.10417079 -16.39201535
  -16.39201535]
 ...
 [-17.03174158 -17.03174158 -16.99398193 ... -17.04699235 -16.44986589
  -16.44986589]
 [-17.03174158 -17.03174158 -16.99398193 ... -17.04699235 -16.44986589
  -16.44986589]
 [-17.03174158 -17.03174158 -16.99398193 ... -17.04699235 -16.44986589
  -16.44986589]]
[[187 187 185 ... 177 173 173]
 [187 187 185 ... 177 173 173]
 [187 187 185 ... 177 173 173]
 ...
 [165 165 165 ... 165 172 172]
 [165 165 165 ... 165 172 172]
 [165 165 165 ... 165 172 172]]
(100, 100) [[-16.39201535 -16.39201535 -17.30842784 ... -17.80872813 -19.10372957
  -19.10372957]
 [-16.39201535 -16.39201535 -17.30842784 ... -17.80872813 -19.10372957
  -19.10372957]
 [-16.39201535 -16.39201535 -17.30842784 ... -17.80872813 -19.103729

(100, 100) [[-16.15897158 -16.15897158 -16.28772391 ... -16.06282482 -15.99951007
  -15.99951007]
 [-16.15897158 -16.15897158 -16.28772391 ... -16.06282482 -15.99951007
  -15.99951007]
 [-16.15897158 -16.15897158 -16.28772391 ... -16.06282482 -15.99951007
  -15.99951007]
 ...
 [-16.72711416 -16.72711416 -14.56090997 ... -17.3224727  -17.17244072
  -17.17244072]
 [-16.72711416 -16.72711416 -14.56090997 ... -17.3224727  -17.17244072
  -17.17244072]
 [-16.72711416 -16.72711416 -14.56090997 ... -17.3224727  -17.17244072
  -17.17244072]]
[[176 176 174 ... 177 178 178]
 [176 176 174 ... 177 178 178]
 [176 176 174 ... 177 178 178]
 ...
 [169 169 196 ... 161 163 163]
 [169 169 196 ... 161 163 163]
 [169 169 196 ... 161 163 163]]
(100, 100) [[-15.99951007 -15.99951007 -17.17505529 ... -14.46865922 -13.68419811
  -13.68419811]
 [-15.99951007 -15.99951007 -17.17505529 ... -14.46865922 -13.68419811
  -13.68419811]
 [-15.99951007 -15.99951007 -17.17505529 ... -14.46865922 -13.68419811
  -13.6841981

(100, 100) [[-14.23703527 -14.23703527 -14.96899518 ... -13.58813914 -13.11401561
  -13.11401561]
 [-14.23703527 -14.23703527 -14.96899518 ... -13.58813914 -13.11401561
  -13.11401561]
 [-14.23703527 -14.23703527 -14.96899518 ... -13.58813914 -13.11401561
  -13.11401561]
 ...
 [-15.2003995  -15.2003995  -13.69447031 ... -18.30386245 -18.34118996
  -18.34118996]
 [-15.2003995  -15.2003995  -13.69447031 ... -18.30386245 -18.34118996
  -18.34118996]
 [-15.2003995  -15.2003995  -13.69447031 ... -18.30386245 -18.34118996
  -18.34118996]]
[[200 200 191 ... 209 215 215]
 [200 200 191 ... 209 215 215]
 [200 200 191 ... 209 215 215]
 ...
 [188 188 207 ... 149 148 148]
 [188 188 207 ... 149 148 148]
 [188 188 207 ... 149 148 148]]
(100, 100) [[-17.15446916 -17.15446916 -17.6688023  ... -18.07268989 -16.54928829
  -16.54928829]
 [-17.15446916 -17.15446916 -17.6688023  ... -18.07268989 -16.54928829
  -16.54928829]
 [-17.15446916 -17.15446916 -17.6688023  ... -18.07268989 -16.54928829
  -16.5492882

[[176 176 193 ... 190 177 177]
 [176 176 193 ... 190 177 177]
 [176 176 193 ... 190 177 177]
 ...
 [181 181 185 ... 151 155 155]
 [181 181 185 ... 151 155 155]
 [181 181 185 ... 151 155 155]]
(100, 100) [[-16.09549454 -16.09549454 -17.7264735  ... -17.16631808 -14.97900928
  -14.97900928]
 [-16.09549454 -16.09549454 -17.7264735  ... -17.16631808 -14.97900928
  -14.97900928]
 [-16.09549454 -16.09549454 -17.7264735  ... -17.16631808 -14.97900928
  -14.97900928]
 ...
 [-17.77145994 -17.77145994 -16.43202515 ... -15.03748653 -16.25750293
  -16.25750293]
 [-17.77145994 -17.77145994 -16.43202515 ... -15.03748653 -16.25750293
  -16.25750293]
 [-17.77145994 -17.77145994 -16.43202515 ... -15.03748653 -16.25750293
  -16.25750293]]
[[177 177 156 ... 163 191 191]
 [177 177 156 ... 163 191 191]
 [177 177 156 ... 163 191 191]
 ...
 [155 155 172 ... 190 175 175]
 [155 155 172 ... 190 175 175]
 [155 155 172 ... 190 175 175]]
(100, 100) [[-14.97900928 -14.97900928 -12.34330671 ... -12.83613878 -12.9674

[[147 147 169 ... 200 202 202]
 [147 147 169 ... 200 202 202]
 [147 147 169 ... 200 202 202]
 ...
 [209 209 208 ... 179 189 189]
 [209 209 208 ... 179 189 189]
 [209 209 208 ... 179 189 189]]
(100, 100) [[-14.08904438 -14.08904438 -14.90756068 ... -16.11945937 -15.45790549
  -15.45790549]
 [-14.08904438 -14.08904438 -14.90756068 ... -16.11945937 -15.45790549
  -15.45790549]
 [-14.08904438 -14.08904438 -14.90756068 ... -16.11945937 -15.45790549
  -15.45790549]
 ...
 [-15.11713768 -15.11713768 -14.33342934 ... -16.22792826 -15.71016816
  -15.71016816]
 [-15.11713768 -15.11713768 -14.33342934 ... -16.22792826 -15.71016816
  -15.71016816]
 [-15.11713768 -15.11713768 -14.33342934 ... -16.22792826 -15.71016816
  -15.71016816]]
[[202 202 192 ... 176 185 185]
 [202 202 192 ... 176 185 185]
 [202 202 192 ... 176 185 185]
 ...
 [189 189 199 ... 175 182 182]
 [189 189 199 ... 175 182 182]
 [189 189 199 ... 175 182 182]]
(100, 100) [[-15.45790549 -15.45790549 -15.9737667  ... -14.19002454 -13.2704

(100, 100) [[-13.77745227 -13.77745227 -14.28467153 ... -14.45324231 -14.15591112
  -14.15591112]
 [-13.77745227 -13.77745227 -14.28467153 ... -14.45324231 -14.15591112
  -14.15591112]
 [-13.77745227 -13.77745227 -14.28467153 ... -14.45324231 -14.15591112
  -14.15591112]
 ...
 [-15.91816643 -15.91816643 -18.27789818 ... -13.71335201 -14.70458829
  -14.70458829]
 [-15.91816643 -15.91816643 -18.27789818 ... -13.71335201 -14.70458829
  -14.70458829]
 [-15.91816643 -15.91816643 -18.27789818 ... -13.71335201 -14.70458829
  -14.70458829]]
[[206 206 200 ... 198 202 202]
 [206 206 200 ... 198 202 202]
 [206 206 200 ... 198 202 202]
 ...
 [179 179 149 ... 207 195 195]
 [179 179 149 ... 207 195 195]
 [179 179 149 ... 207 195 195]]
(100, 100) [[-14.15591112 -14.15591112 -14.05118099 ... -13.06144214 -13.41318654
  -13.41318654]
 [-14.15591112 -14.15591112 -14.05118099 ... -13.06144214 -13.41318654
  -13.41318654]
 [-14.15591112 -14.15591112 -14.05118099 ... -13.06144214 -13.41318654
  -13.4131865

(100, 100) [[-17.71016558 -17.71016558 -17.42640223 ... -19.16184928 -18.41774607
  -18.41774607]
 [-17.71016558 -17.71016558 -17.42640223 ... -19.16184928 -18.41774607
  -18.41774607]
 [-17.71016558 -17.71016558 -17.42640223 ... -19.16184928 -18.41774607
  -18.41774607]
 ...
 [-17.52827915 -17.52827915 -18.03758272 ... -17.09561017 -17.84652879
  -17.84652879]
 [-17.52827915 -17.52827915 -18.03758272 ... -17.09561017 -17.84652879
  -17.84652879]
 [-17.52827915 -17.52827915 -18.03758272 ... -17.09561017 -17.84652879
  -17.84652879]]
[[156 156 160 ... 138 147 147]
 [156 156 160 ... 138 147 147]
 [156 156 160 ... 138 147 147]
 ...
 [159 159 152 ... 164 154 154]
 [159 159 152 ... 164 154 154]
 [159 159 152 ... 164 154 154]]
(100, 100) [[-18.41774607 -18.41774607 -18.29301269 ... -18.15208915 -19.45665572
  -19.45665572]
 [-18.41774607 -18.41774607 -18.29301269 ... -18.15208915 -19.45665572
  -19.45665572]
 [-18.41774607 -18.41774607 -18.29301269 ... -18.15208915 -19.45665572
  -19.4566557

In [2]:
from pynvml import *
nvmlInit()
try:
    deviceCount = nvmlDeviceGetCount()
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))
except NVMLError as error:
    print(error)

Device 0 : b'Tesla P100-PCIE-12GB'


In [3]:
#looking at tests from https://docs.fast.ai/dev/gpu.html

In [4]:
import torch
t_cpu = torch.rand(500,500,500)
%timeit t_cpu @ t_cpu

325 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
t_gpu = torch.rand(500,500,500).cuda()
%timeit t_gpu @ t_gpu

15.5 ms ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.2.0 =====================================
  Version control:               4.2.0

  Platform:
    Timestamp:                   2020-04-04T14:19:38Z
    Host:                        Linux 4.15.0-1028-gcp x86_64
    CMake:                       3.9.0
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (14 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (4 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX

  C/C++:
    Built as dynamic libs?:      NO
    C++ Compiler:                /usr/lib/ccache/com

In [6]:
from fastai2.vision.all import *
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

def sigmoid(x):
    return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
def softmax(x):
    z = x - x.max()
    return np.exp(z)/np.exp(z).sum()

model = resnet34
use_bce = True